In [1]:
import re
import requests
from bs4 import BeautifulSoup
import time
from urllib import parse


# 사이드 메뉴 열 필요 없어서 일땐 빼놈. @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# 여기 참고해서 작성해봤었음. https://brunch.co.kr/@jk-lab/18




# 지역별로 버튼 클릭할 필요없이 걍 다 담겨있네..
# from selenium import webdriver

# chromedriver_dir = '/home/nasanmaro/Desktop/projects/yapen/test/selenium_crawling_test/chromedriver'
# driver = webdriver.Chrome(chromedriver_dir)
# driver.get('http://www.yapen.co.kr')
# time.sleep(5)

# # 왼쪽 메뉴를 연다. 
# loca = driver.find_element_by_class_name('yapenMenu')
# loca.click()
# time.sleep(5)

# # 드라이버 인스턴스로부터 현제 메뉴 연상태의 페이지 소스 받아서 source에 넣는다. 
# source = driver.page_source

# # soup 객체로 만듬.
# soup = BeautifulSoup(source, 'lxml')



# 이것까지 하면 사이드 메뉴여는것 됬다. 
##########뒤에 아직따라안한 부분.################################################


# sido = driver.find_element_by_class_name('sido_area_box')
# li = sido.find_elements_by_tag_name('li')
# li[0].click()
# time.sleep(5)

# gugun = driver.find_element_by_class_name('gugun_area_box')
# guli = gugun.find_element_by_tag_name('li')
# guuli.click()
# time.sleep(5)


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@









#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# location_name_list 뽑는 과정 

def location_name_list_crawler():
    request = requests.get("http://www.yapen.co.kr")
    response = request.text
    soup = BeautifulSoup(response, 'lxml')

    left_menu=soup.select('div.locLayer')
    # 풀빌라, MD추천 제외 14지역중 7지역 만남김.
    selected_left_menu= left_menu[2:9]
    

    # 여기에 list 형태로 지역,지역고유번호/(세부지역,고유번호) 넣고싶다. 
    # location_name_list =[ [지역1 ,지역1고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],
    #                       [지역2 ,지역2고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],..
    location_name_list=list()

    for location in selected_left_menu:
        # 지역 이름 먼저 뽑음 
        location_name = location.select('div.titleStyle')[0].get_text(strip=True)
        location_name_sub_list=list()
        location_name_sub_list.append(location_name)

        li=location.select('li')

        #for문 돌면서 (고유번호 세부지역) 리스트에 담은뒤 location_list에 넣겠다.
        location_detail_list=[] 
        for location_detail in li:
            onclick_value = location_detail['onclick']  #regionMove('1.003021','금산/논산');
            
            split_right = onclick_value.split(',')[0]
            split_left = onclick_value.split(',')[1]

            sub_location_no = re.findall("'(.+)'",split_right)[0]
            sub_location_name = re.findall("'(.+)'",split_left)[0]

            location_detail_tuple = (sub_location_no,sub_location_name)

             # (고유번호/세부지역) 리스트에 담음
            location_detail_list.append(location_detail_tuple)
        
        # 지역 고유번호부터 정규표현식으로 뽑아내서 담음.(세부지역 고유번호의 소숫점 뒤 3자리)
        
        location_detail_no_for_search = location_detail_list[0][0] # '1.003021'
        location_detaol_no = re.findall(".(\d\d\d)",location_detail_no_for_search)[0]
        location_name_sub_list.append(location_detaol_no)
        
        # location_name_sub_list (고유번호 세부지역) 리스트를 넣음.
        location_name_sub_list.append(location_detail_list)
        
        # 상위 리스트에 넣음 
        location_name_list.append(location_name_sub_list)
        
    return location_name_list
    
    
# location_dict 뽑는 과정 끝
######################################################################################












#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#이제 각 페이지 location 가서 해커톤때 쓴 pension crawler 돌리고 싶다.
# 메인페이지에서 기본정보 3개만 여러개 팬션에게서 가져왔던것.


#     url = "http://www.yapen.co.kr/region?location=015&subLocation=1.015001"
def pension_crawler(location_no,sub_location_no):

    params = {
            'location':location_no,
             'subLocation':sub_location_no,
        }

    url = "http://www.yapen.co.kr/region?" + parse.urlencode(params)

    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')


    title_list=list()
    img_file_list=list()
    price_list=list()
    pldx_list=list()

    title_uls = soup.select('ul.dest-place-opt-fea')
    for ul in title_uls:
        li=ul.select('li')
        title_list.append(li[1].get_text())

    price_uls = soup.select('ul.dest-place-opt-cast')
    for ul in price_uls:
        li=ul.select('li')
        price_list.append(li[1].get_text())

    
    img_file_divs = soup.select('div.imgBox')
    for div in img_file_divs:
        img_file_list.append(div.select('img')[0]['src'])
        
        list1 = re.split('/', div.select('img')[0]['src'])
        pldx_list.append(int(list1[5]))


    
    sub_locations_info_list = [title_list, price_list, img_file_list,pldx_list]
    
    return sub_locations_info_list

###########################################################################################









###########################################################################################
##province_name_list 로부터 지역명,고유번호 받아서 각 세부지역별로 pension_crawler 
## 해서 기본정보 3개씩 모으는 크롤러##

def location_crawler():
    
    location_info_list=list()
    
    location_name_list = location_name_list_crawler()
    # location_name_list =[ [지역1 ,지역1고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],
    #                       [지역2 ,지역2고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],..
    
    for location in location_name_list:
        location_name = location[0]
        location_no = location[1]
        sub_location_list = location[2]
        for sub_location in sub_location_list:
            print(sub_location)
            sub_location_no = sub_location[0]
            sub_location_name = sub_location[1]
            sub_locations_info_list = pension_crawler(location_no, sub_location_no)
            
            #기존에 pension모델 1차적으로 이름,가격,이미지로  만들던것에
            #location, sub_location 속성 추가해서 이곳에서 만들면 될듯하다.
            # 일단은 리스트 형태로 정보 6개 묶어서 저장해보겠음.
            #[[location,sub_location,title,pricem,img_file,pldx]....팬션 999까지 한 리스트에
            
            for i in range(len(sub_locations_info_list[0])):
                location_info_list.append([location_name,
                                           sub_location_name,
                                           sub_locations_info_list[0][i],# title
                                           sub_locations_info_list[1][i],# price
                                           sub_locations_info_list[2][i],# img_file
                                           sub_locations_info_list[3][i],# pldx
                                          ])
    
    return location_info_list
    











#  해커톤때 모델저장하는것 크롤러에서 다해버렸는데 추후에 여기 참고하면 좋을듯.


#  params = {
#         'ypIdx': ypIdx
#     }

#     url = "http://www.yapen.co.kr/details?" + parse.urlencode(params)

#     request = requests.get(url)
#     response = request.text
#     soup = BeautifulSoup(response, 'lxml')

#     name_root = soup.select('div.wrap_1000')
#     name = name_root[0].select('h3')[0].get_text()

#     table = soup.select('table.pensionTbl')
#     trs = table[0].select('tr')
#     tds = trs[0].select('td')
#     address = tds[0].get_text()

#     tds2 = trs[1].select('td')
#     check_in_out = tds2[0].select('span')
#     check_in = check_in_out[0].get_text()
#     check_out = check_in_out[1].get_text()

#     td3 = trs[3].select('td')
#     number_tags = td3[0].select('span')
#     room = number_tags[0].get_text()

#     td4 = trs[4].select('td')
#     infos = td4[0].select('p')
#     info_result = ''
#     for info in infos:
#         info_result = info_result + '\n' + info.get_text() + '\n'

#     td5 = trs[5].select('td')
#     lis = td5[0].select('li')
#     theme_result = ''
#     for li in lis:
#         theme_result = theme_result + '\n' + li.get_text() + '\n'




#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@실행창 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# province_name_list = province_name_list_crawler()
# print(province_name_list)

test = location_name_list_crawler()

print(test)
# pension_crawler('015','1.015001')

# province_crawler()


location_info_list = location_crawler()
print(location_info_list)





[['가평', '015', [('1.015001', '가평 전체'), ('1.015002', '남이섬/자라섬/가평북부'), ('1.015003', '아침고요수목원/쁘띠프랑스/청평')]], ['경기', '001', [('1.001010', '양평'), ('1.001005', '대부도'), ('1.001016', '포천'), ('1.001006', '남양주'), ('1.001015', '파주/양주/김포'), ('1.001025', '연천/동두천'), ('1.001020', '용인/안성/평택'), ('1.001019', '여주/광주/이천'), ('1.001017', '화성(제부도)'), ('1.001022', '부천')]], ['강원', '002', [('1.002013', '춘천(남이섬)'), ('1.002020', '속초/고성'), ('1.002001', '강릉'), ('1.002015', '홍천'), ('1.002014', '평창'), ('1.002007', '양양'), ('1.002017', '횡성'), ('1.002023', '영월/원주'), ('1.002022', '삼척/동해/정선'), ('1.002021', '화천/철원/인제/양구')]], ['제주', '009', [('1.009001', '제주시 전체'), ('1.009002', '서귀포시 전체'), ('1.009004', '└ 제주/애월/한림/한경'), ('1.009006', '└ 서귀포/중문/대정/안덕'), ('1.009005', '└ 조천/구좌/우도'), ('1.009007', '└ 남원/표선/성산')]], ['인천', '010', [('1.010001', '강화도'), ('1.010007', '석모도'), ('1.010009', '을왕리/무의도'), ('1.010004', '옹진')]], ['충남', '003', [('1.003013', '태안(안면도)'), ('1.003018', '보령/서천'), ('1.003005', '서산/당진'), ('1.003019', '홍성/예산/천안/아산'), ('

In [ ]:

#  province_dict  뽑는 과정인데 list로 바꾼후 여기다 일단 저장해 놓음. 나중에 필요하면 가져다 써라.




#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# province_dict 뽑는 과정 

def province_dict_crawler():
    request = requests.get("http://www.yapen.co.kr")
    response = request.text
    soup = BeautifulSoup(response, 'lxml')

    left_menu=soup.select('div.locLayer')
    # 풀빌라, MD추천 제외 14지역중 7지역 만남김.
    selected_left_menu= left_menu[2:9]
    # print(selected_left_menu)

    # 여기에 dict 형태로 지역/(세부지역,고유번호) 넣고싶다. 
    # location_dict = {'지역':[(고유번호,세부지역),(고유번호2,세부지역2),(고유번호3,세부지역3)]}
    location_dict=dict()

    for location in selected_left_menu:
        # 지역 이름 먼저 뽑음 
        province_title = location.select('div.titleStyle')[0].get_text(strip=True)
    #     print(province_title)

        li=location.select('li')

        #for문 돌면서 (고유번호 세부지역) 리스트에 담은뒤 province_dict에 넣겠다.
        location_detail_list=[] 
        for location_detail in li:
            onclick_value = location_detail['onclick']  #regionMove('1.003021','금산/논산');  
            location_detail_tuple = tuple(re.findall("'(.+)'",onclick_value))

             # (고유번호 세부지역) 리스트에 담음
            location_detail_list.append(location_detail_tuple)

        # location_dict에 지역과 함께 (고유번호 세부지역) 리스트에 넣음.
        location_dict[province_title]= location_detail_list
        
    return location_dict
    
    
# province_dict 뽑는 과정 끝
######################################################################################